In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [ ]:
E = 68*(10**9) # modulus of rigidity
density = 2710 # density of aluminium
neu = 0.31 #poisson's ratio
h=0.002 #thickness of plate
V = h* 6* 6
m = density * V
A = 6 * 6
rho = m/A
D = E*(h**3)/(12*(1-neu**2)) #Flexural rigidity of flat plate

In [ ]:
class PlateVibration:
    def __init__(self, X, Y, T, X_b, Y_b, T_b):
        self.x = torch.tensor(X, dtype=torch.float32, requires_grad=True)
        self.y = torch.tensor(Y, dtype=torch.float32, requires_grad=True)
        self.t = torch.tensor(T, dtype=torch.float32, requires_grad=True)
        self.x_b = torch.tensor(X_b, dtype=torch.float32, requires_grad=True)
        self.y_b = torch.tensor(Y_b, dtype=torch.float32, requires_grad=True)
        self.t_b = torch.tensor(T_b, dtype=torch.float32, requires_grad=True)

        self.null = torch.zeros((self.x.shape[0], 1))
        self.network()

        self.optimizer = torch.optim.LBFGS(self.net.parameters(), lr=1, max_iter=200000, max_eval=50000,
                                           history_size=50, tolerance_grad=1e-05, tolerance_change=0.5 * np.finfo(float).eps,
                                           line_search_fn="strong_wolfe")

        self.mse = nn.MSELoss()
        self.ls = 0
        self.iter = 0

    def network(self):
        self.net = nn.Sequential(
            nn.Linear(3, 20), nn.Tanh(),
            nn.Linear(20, 20), nn.Tanh(),
            nn.Linear(20, 20), nn.Tanh(),
            nn.Linear(20, 20), nn.Tanh(),
            nn.Linear(20, 20), nn.Tanh(),
            nn.Linear(20, 20), nn.Tanh(),
            nn.Linear(20, 20), nn.Tanh(),
            nn.Linear(20, 1)
        )

    def function(self, x, y, t):
        u_pred = self.net(torch.hstack((x, y, t)))

        u_pred_x = torch.autograd.grad(u_pred, x, grad_outputs=torch.ones_like(u_pred), create_graph=True, allow_unused=True)[0]
        u_pred_y = torch.autograd.grad(u_pred, y, grad_outputs=torch.ones_like(u_pred), create_graph=True, allow_unused=True)[0]
        u_pred_xx = torch.autograd.grad(u_pred_x, x, grad_outputs=torch.ones_like(u_pred_x), create_graph=True, allow_unused=True)[0]
        u_pred_xxx = torch.autograd.grad(u_pred_xx, x, grad_outputs=torch.ones_like(u_pred_x), create_graph=True, allow_unused=True)[0]
        u_pred_yy = torch.autograd.grad(u_pred_y, y, grad_outputs=torch.ones_like(u_pred_y), create_graph=True, allow_unused=True)[0]
        u_pred_yyy = torch.autograd.grad(u_pred_yy, y, grad_outputs=torch.ones_like(u_pred_y), create_graph=True, allow_unused=True)[0]
        u_pred_xxxx = torch.autograd.grad(u_pred_xxx, x, grad_outputs=torch.ones_like(u_pred_x), create_graph=True, allow_unused=True)[0]
        u_pred_yyyy = torch.autograd.grad(u_pred_yyy, y, grad_outputs=torch.ones_like(u_pred_y), create_graph=True, allow_unused=True)[0]
        u_pred_xxy = torch.autograd.grad(u_pred_xx, y, grad_outputs=torch.ones_like(u_pred_xx), create_graph=True, allow_unused=True)[0]
        u_pred_xxyy = torch.autograd.grad(u_pred_xxy, y, grad_outputs=torch.ones_like(u_pred_xxy), create_graph=True, allow_unused=True)[0]

        u_pred_t = torch.autograd.grad(u_pred, t, grad_outputs=torch.ones_like(u_pred), create_graph=True, allow_unused=True)[0]
        u_pred_tt = torch.autograd.grad(u_pred_t, t, grad_outputs=torch.ones_like(u_pred_t), create_graph=True, allow_unused=True)[0]

        f = D * (u_pred_xxxx + u_pred_yyyy + 2 * u_pred_xxyy) + rho * u_pred_tt

        return u_pred, f

    def boundary_conditions(self, x_b, y_b, t_b):
        u_boundary_pred = self.net(torch.hstack((x_b, y_b, t_b)))

        boundary_grad_x = torch.autograd.grad(u_boundary_pred, x_b, grad_outputs=torch.ones_like(u_boundary_pred), create_graph=True, allow_unused=True)[0]
        boundary_grad_xx = torch.autograd.grad(boundary_grad_x, x_b, grad_outputs=torch.ones_like(boundary_grad_x), create_graph=True, allow_unused=True)[0]

        boundary_grad_y = torch.autograd.grad(u_boundary_pred, y_b, grad_outputs=torch.ones_like(u_boundary_pred), create_graph=True, allow_unused=True)[0]
        boundary_grad_yy = torch.autograd.grad(boundary_grad_y, y_b, grad_outputs=torch.ones_like(boundary_grad_y), create_graph=True, allow_unused=True)[0]

        return u_boundary_pred, boundary_grad_xx, boundary_grad_yy

    def closure(self):
        self.optimizer.zero_grad()

        u_pred, f_pred = self.function(self.x, self.y, self.t)

        b, b_grad_xx, b_grad_yy = self.boundary_conditions(self.x_b, self.y_b, self.t_b)

        null_b = torch.zeros_like(b)
        null_b_grad_xx = torch.zeros_like(b_grad_xx)
        null_b_grad_yy = torch.zeros_like(b_grad_yy)

        null_f_pred = torch.zeros_like(f_pred)

        b_loss = self.mse(b, null_b)
        b_grad_loss_xx = self.mse(b_grad_xx, null_b_grad_xx)
        b_grad_loss_yy = self.mse(b_grad_yy, null_b_grad_yy)

        f_loss = self.mse(f_pred, null_f_pred)

        self.ls = b_loss + b_grad_loss_xx + b_grad_loss_yy +f_loss
        self.ls.backward()

        self.iter += 1
        if not self.iter % 1:
            print(f'Iteration: {self.iter}, Loss: {self.ls.item():.6f}')

        return self.ls

    def train(self):
        self.net.train()
        self.optimizer.step(self.closure)


In [ ]:
N_interior = 30

# Interior points
x_interior = torch.linspace(0, 1, N_interior).float()
y_interior = torch.linspace(0, 1, N_interior).float()
t_star = torch.linspace(0,20, 200).float()

# Creating a meshgrid
xx, yy = torch.meshgrid(x_interior, y_interior, indexing='ij')

# Flattening the meshgrid
x_star = xx.flatten()[:, None]
y_star = yy.flatten()[:, None]

N = x_star.shape[0]
T = t_star.shape[0]

XX = np.tile(x_star, (1, T))  # N x T
YY = np.tile(y_star, (1, T))  # N x T
TT = np.tile(t_star, (1, N)).T  # N x T

x = XX.flatten()[:, None]  # NT x 1
y = YY.flatten()[:, None]  # NT x 1
t = TT.flatten()[:, None]  # NT x 1

# Creating boundary points
x_b_star = x_interior.flatten()[:,None]
y_b_star = y_interior.flatten()[:,None]

x0 = torch.full_like(x_b_star, 0)
x2 = torch.full_like(x_b_star, 1)
y0 = torch.full_like(y_b_star, 0)
y2 = torch.full_like(y_b_star, 1)

x_boundary = torch.cat((x_b_star, x_b_star, x0, x0), dim=0)
y_boundary = torch.cat((y0, y0, y_b_star, y_b_star), dim=0)

N_b = x_boundary.shape[0]

xx_b = np.tile(x_boundary, (1, T))  # N x T
yy_b = np.tile(y_boundary, (1, T))  # N x T
tt_b = np.tile(t_star, (1, N_b)).T  # N x T

x_b = xx_b.flatten()[:, None]  # NT x 1
y_b = yy_b.flatten()[:, None]  # NT x 1
t_b = tt_b.flatten()[:, None]  # NT x 1

pinn = PlateVibration(x, y, t, x_b, y_b, t_b)
pinn.train()